In [1]:
from el_toolkit.document import Document
from el_toolkit.lkb.lexical_knowledge_base import Lexical_Knowledge_Base
from transformers import BertTokenizer


lkb = Lexical_Knowledge_Base.read_json("data/knowledge_datasets/mesh_subset.json")

biomed_tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path='monologg/biobert_v1.1_pubmed',
                                            do_lower_case=False, cache_dir=None)
                                            
doc = Document.read_json("/Users/coltonflowers/Work/TREND_repos/BioMedical-EL/tests/test_data/test_docs/without_overlaps/14_mentions.json")
doc.jupyter_display(lkb)
for doc in doc.segment(biomed_tokenizer,8):
  doc.jupyter_display(lkb)
  print("-----------------------------------")

Mentions Omitted: 0


-----------------------------------


-----------------------------------


In [5]:
from el_toolkit.entity_linkers.dual_embedder.featurizer import DualEmbedderTrainFeaturizer
from el_toolkit.entity_linkers.dual_embedder.entity_linker import BertConceptEmbedder,DocumentEmbedder,DualEmbedderEntityLinker

concept_embedder = BertConceptEmbedder(lkb,model=None,tokenizer=biomed_tokenizer,max_seq_len=256)
document_embedder = DocumentEmbedder(model=None,tokenizer = biomed_tokenizer,max_seq_len=256)
dual_embedder = DualEmbedderEntityLinker(concept_embedder,document_embedder)
featurizer = DualEmbedderTrainFeaturizer(dual_embedder,lkb=lkb,num_hard_negatives=0,num_random_negatives=8,num_max_mentions=8)


In [40]:

from el_toolkit.document import Document
doc = Document.read_json("/Users/coltonflowers/Work/TREND_repos/BioMedical-EL/tests/test_data/test_docs/without_overlaps/14_mentions.json")
doc_tokens,doc_tokens_mask,mention_start_markers,mention_end_markers,sequence_tags,too_long = document_embedder.encode_document(doc)

In [3]:
docs = doc.segment(biomed_tokenizer,8)
dataset = featurizer.featurize(docs)

Mentions Omitted: 0


UnboundLocalError: local variable 'bio_tags' referenced before assignment